In [169]:
import pandas as pd
from sklearn.neighbors import BallTree
import geopy.distance
import numpy as np
import sqlite3
import math

In [84]:
df = pd.read_csv("../data/collated_hawkercentre_data.csv")
df = df.drop(columns=["Unnamed: 0"])

In [85]:
df.head(n=2)

,LONGITUDE,LATITUDE,ADDRESSBLOCKHOUSENUMBER,EST_ORIGINAL_COMPLETION_DATE,STATUS,LANDYADDRESSPOINT,PHOTOURL,DESCRIPTION,NAME,ADDRESSTYPE,...,INC_CRC,FMEL_UPD_D,APPROXIMATE_GFA,AWARDED_DATE,ADDRESSBUILDINGNAME,IMPLEMENTATION_DATE,CLEANINGSTARTDATE,INFO_ON_CO_LOCATORS,CLEANINGENDDATE,RNR_STATUS
0,103.938733,1.331987,85,30/6/1977,Existing,34910.13,http://www.nea.gov.sg/images/default-source/Ha...,HUP Standard Upgrading,Bedok North Street 4 Blk 85 (85 Fengshan Centre),I,...,CFC780A1B5DC7721,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,103.818339,1.287331,85,4/4/1972,Existing,29972.02,http://www.nea.gov.sg/images/default-source/Ha...,HUP Reconfiguration,Redhill Lane Blk 85 (Redhill Food Centre),I,...,1D515CA502CE0A60,20210330151704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
test_locations = [[np.random.uniform(1.31, 1.35), np.random.uniform(103.3, 103.9)] for i in range(1000)]

### Testing with brute force method - in memory python

This method will be the most accurate one as all points are visited before determining the nearest neighbours.

In [147]:
lat_lon_list = df[["LATITUDE", "LONGITUDE"]].values.tolist()
hawkercentre_list = df["NAME"].tolist()

In [172]:
results = {hawkercentre_list[i]: geopy.distance.geodesic(test_locations[0], lat_lon_list[i]).km for i in range(len(hawkercentre_list))}
sorted(results, key=results.get)[:5]

['Jurong West Hawker Centre',
 'Boon Lay Place Blk 221A/B (Boon Lay Place Market and Food Village)',
 'Jurong West Street 52 Blk 505',
 'Taman Jurong Market and Food Centre',
 'Jurong East Ave 1 Blk 347 (Yuhua Market and Hawker Centre)']

In [96]:
%%timeit
# test latency
results = {hawkercentre_list[i]: geopy.distance.geodesic(test_locations[0], lat_lon_list[i]).km for i in range(len(hawkercentre_list))}
sorted(results, key=results.get)[:5]

22.6 ms ± 3.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [104]:
results = []
for test_location in test_locations:
    temp_dist = {hawkercentre_list[i]: geopy.distance.geodesic(test_location, lat_lon_list[i]).km for i in range(len(hawkercentre_list))}
    results.append(sorted(temp_dist, key=temp_dist.get)[:5])

### Testing with brute force method - SQL via sql lite db

This method will be the most accurate one as all points are visited before determining the nearest neighbours.

In [118]:
def get_cos_radians(x: float) -> float:
    return math.cos(math.radians(x))

def get_radians(x: float) -> float:
    return math.radians(x)

def get_sin_radians(x: float) -> float:
    return math.sin(math.radians(x))

In [119]:
# pre-calculate cos and sin radians
df["LATITUDE_RADIANS_COS"] = df["LATITUDE"].apply(get_cos_radians)
df["LONGITUDE_RADIANS"] = df["LONGITUDE"].apply(get_radians)
df["LATITUDE_RADIANS_SIN"] = df["LATITUDE"].apply(get_sin_radians)

In [175]:
# set up database and create table
conn = sqlite3.connect('test_database')
c = conn.cursor()

In [176]:
# dump dataframe details into table
df[["NAME", 
    "PHOTOURL", 
    "LATITUDE", 
    "LONGITUDE",
    "LATITUDE_RADIANS_SIN",
    "LATITUDE_RADIANS_COS",
    "LONGITUDE_RADIANS"]].to_sql("hawkercentre", conn, if_exists="replace", index = False)

125

In [177]:
# distance function for sqlite
def get_distance(input_latitude, 
             input_longitude, 
             latitude_radians_cos, 
             longitude_radians, 
             latitude_radians_sin):
    # multiplier for distance in km
    multiplier = 6371 
    return multiplier * \
        math.acos(
            math.cos(math.radians(input_latitude)) *
            latitude_radians_cos *
            math.cos(longitude_radians - math.radians(input_longitude) ) +
            math.sin(math.radians(input_latitude)) * latitude_radians_sin
        )
    
conn.create_function("get_distance", 5, get_distance)

# allow for debugging
sqlite3.enable_callback_tracebacks(True) 

In [178]:
input_lat = test_locations[0][0]
input_lon = test_locations[0][1]

In [181]:
query = f'''  
    SELECT 
        name,
        photourl
    FROM hawkercentre
    ORDER BY get_distance(
        {input_lat},
        {input_lon},
        latitude_radians_cos,
        longitude_radians,
        latitude_radians_sin)
    LIMIT 5
'''

In [182]:
c.execute(query)
result = c.fetchall()
print(result)

[('Jurong West Hawker Centre', 'http://www.nea.gov.sg/images/default-source/Hawker-Centres-Division/jurong-west-hawker-centre.jpg'), ('Boon Lay Place Blk 221A/B (Boon Lay Place Market and Food Village)', 'http://www.nea.gov.sg/images/default-source/Hawker-Centres-Division/resize_1262154766447.jpg'), ('Jurong West Street 52 Blk 505', 'http://www.nea.gov.sg/images/default-source/Hawker-Centres-Division/resize_1262155040710.jpg'), ('Taman Jurong Market and Food Centre', 'http://www.nea.gov.sg/images/default-source/Hawker-Centres-Division/resize_1262156147897.jpg'), ('Jurong East Ave 1 Blk 347 (Yuhua Market and Hawker Centre)', 'http://www.nea.gov.sg/images/default-source/Hawker-Centres-Division/resize_1275537901886.jpg')]


In [168]:
%%timeit
c.execute(query)
result = c.fetchall()

1.78 ms ± 411 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Testing with Sklearn Ball Tree Algorithm

In [98]:
ball_tree = BallTree(df[["LATITUDE", "LONGITUDE"]], metric="haversine")

In [138]:
%%timeit
# test latency
ball_query_results = ball_tree.query(
    [test_locations[0]],
    k=5,
    return_distance=False,
    sort_results=True,
)

32.6 µs ± 665 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [130]:
ball_query_results = ball_tree.query(
    test_locations,
    k=5,
    return_distance=False,
    sort_results=True,
)
ball_query_results = [df["NAME"].iloc[r].tolist() for r in ball_query_results]

### Conclusion

In [133]:
# calculating accuracy using Mean Average Precision @ 5
# ground truth is brute force

individual_MAP = [len(set(results[i]).intersection(ball_query_results[i]))/5 for i in range(len(results))]
overall_MAP = sum(individual_MAP)/len(individual_MAP)
print(overall_MAP)

0.7581999999999991
